In [1]:
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

from basegame.game import Game
from basegame.player import Player

In [2]:
from benchmarks.simplePlayer import SimplePlayer
from benchmarks.simpleNoCheck import SimpleNoCheck
from benchmarks.simpleAlwaysCheck import SimpleAlwaysCheck
from benchmarks.honestPlayer import HonestPlayer

from moje_dzieci.MyFirstPlayer import MyFirstPlayer
from moje_dzieci.MySecondPlayer import MySecondPlayer
from moje_dzieci.MyThirdPlayer import MyThirdPlayer
from absolutnie_do_wywalenia.cardCounter import FirstCardCounter
from absolutnie_do_wywalenia.cardCounter_base2 import SecondCardCounter
from czyje.Kondys_Dabrowski import Kondys_Dabrowski

In [3]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [4]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(11, 1), (11, 0), (14, 1), (9, 2), (11, 2), (14, 3), (13, 2), (10, 2)]
[(13, 1), (12, 0), (13, 0), (12, 3), (12, 2), (10, 3), (14, 2), (9, 1)]


In [5]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

Analyze few moves...

In [6]:
### Perform a full game 100 times
STAT_NAMES = ["Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"]
ANALYZE_STATS = [0, 1, 2, 3, 5, 6, 7, 8]

def printResults(results):
    print("Wins:")
    print(results[0])
    print("Draws:")
    print(results[1])
    print("Moves:")
    print(results[2])
    print("Cards:")
    print(results[3])
    print("Pile size:")
    print(results[4])
    print("Checks:")
    print(results[5])
    print("Draw decisions:")
    print(results[6])
    print("Cheats:")
    print(results[7])
    print("Errors:")
    print(results[8])
    print("Total errors:")
    print(results[9])

def comparePlayers(player1_class, player2_class):
    stats_wins = [0, 0]
    stats_draws = [0, 0]
    stats_moves = [0, 0]
    stats_cards = [0, 0]
    stats_pile_size = 0
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]
    
    repeats = 1000
    errors = 0
    draws = 0

    for t in range(repeats):
        player1 = player1_class("")
        player2 = player2_class("")
        game = Game([player1, player2], log = False)
    
        error = False
        draw = False
        
        while True:
            valid, player = game.takeTurn(log = False)
            if game.moves[0] > 100 or game.moves[1] > 100:
                draws += 1
                stats_draws[0] += 1
                stats_draws[1] += 1
                if (game.player_cards[0] < game.player_cards[1]):
                    stats_wins[0] += 1
                if (game.player_cards[0] > game.player_cards[1]):
                    stats_wins[1] += 1
                    
                    
                draw=True
                # print("DRAW")
                break
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = False):
                stats_wins[player] += 1
                break
            
        stats_pile_size += len(game.pile)
        if error: continue
        #if draw: continue
       
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

    div = repeats - errors
    if div > 0:
            
        stats_pile_size /= div          
        for j in range(2):
            stats_moves[j] /= div
            stats_cheats[j] /= div
            stats_checks[j] /= div
            stats_draw_decisions[j] /= div
            stats_cards[j] /= div
            
    return [stats_wins, stats_draws, stats_moves, stats_cards, stats_pile_size, stats_checks, 
            stats_draw_decisions, stats_cheats, stats_errors, errors, draws]  


# COMPARE

In [7]:
from moje_dzieci.Kot_4 import Kot4
from moje_dzieci.Kot_5 import Kot5
from moje_dzieci.Kot_6 import Kot6
from moje_dzieci.Kociak import Kociak
from moje_dzieci.Kociak_simple import KociakSimple
from moje_dzieci.Kociak_proba import KociakProba

In [8]:
strategy = [
    ["RandomPlayer", "RANDOM", RandomPlayer],
    ["SimplePlayer", "SimplePlayer", SimplePlayer],
    ["NoCheck", "NoCheck", SimpleNoCheck],
    ["AlwaysCheck", "AlwaysCheck", SimpleAlwaysCheck],
    ["HonestPlayer", "HonestPlayer", HonestPlayer],
    ["Kot_First", "Kot_First", MyFirstPlayer],
    ["Kot_Second", "Kot_Second", MySecondPlayer],
    ["Kot_Third", "Kot_Third", MyThirdPlayer],
    ["kacper","Kacper",Kondys_Dabrowski],
    ["Kot4","Kot4", Kot4],
    ["Kot5","Kot5", Kot5],
    ["Kot6","Kot6", Kot6],
    ["KociakSimple","KociakSimple", KociakSimple],
    ["Kociak_proba","Kociak_proba", KociakProba],
    # ["Kociak","Kociak", Kociak],
   ]

#%pdb on
full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    # print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        # print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        full_results[A][B] = results


def generate_ranking(full_results, strategy):
    n = len(strategy)
    total_wins = [0 for _ in range(n)]

    for i in range(n):
        for j in range(n):
            if i < j and full_results[i][j] is not None:
                wins_i, wins_j = full_results[i][j][0]
                total_wins[i] += wins_i
                total_wins[j] += wins_j

    # Sparuj nazwy z wynikami
    ranking = list(zip([s[0] for s in strategy], total_wins))

    # Posortuj malejąco
    ranking.sort(key=lambda x: x[1], reverse=True)

    maxxx = (len(strategy) -1) * 1000

    print("RANKING GRACZY (według liczby wygranych):")
    for idx, (name, wins) in enumerate(ranking, 1):
        print(f"{idx}. {name} - {wins} / {maxxx} wygranych")

    return ranking

ranking = generate_ranking(full_results, strategy)

RANKING GRACZY (według liczby wygranych):
1. kacper - 11182 / 14000 wygranych
2. Kot4 - 9598 / 14000 wygranych
3. Kot_Second - 9151 / 14000 wygranych
4. Kot6 - 8916 / 14000 wygranych
5. Kot5 - 8749 / 14000 wygranych
6. Kot_Third - 8023 / 14000 wygranych
7. Kociak_proba - 7394 / 14000 wygranych
8. KociakSimple - 7050 / 14000 wygranych
9. NoCheck - 6962 / 14000 wygranych
10. Kot_First - 6752 / 14000 wygranych
11. Kociak - 6686 / 14000 wygranych
12. SimplePlayer - 6077 / 14000 wygranych
13. HonestPlayer - 5089 / 14000 wygranych
14. RandomPlayer - 2108 / 14000 wygranych
15. AlwaysCheck - 1263 / 14000 wygranych


In [9]:
import pandas as pd
import numpy as np

def create_win_matrix(full_results, strategy):
    names = [s[0] for s in strategy]
    n = len(names)
    matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i == j:
                matrix[i][j] = np.nan  # Brak pojedynków z samym sobą
            elif i < j and full_results[i][j] is not None:
                wins_i, wins_j = full_results[i][j][0]
                matrix[i][j] = wins_i
                matrix[j][i] = wins_j

    df = pd.DataFrame(matrix, index=names, columns=names)
    print("MACIERZ WYGRANYCH:")
    print(df.round(1))

    return df

pd.set_option('display.width', 1000)
df = create_win_matrix(full_results, strategy)

MACIERZ WYGRANYCH:
              RandomPlayer  SimplePlayer  NoCheck  AlwaysCheck  HonestPlayer  Kot_First  Kot_Second  Kot_Third  kacper   Kot4   Kot5   Kot6  KociakSimple  Kociak_proba  Kociak
RandomPlayer           NaN         100.0    580.0          8.0           9.0       59.0       124.0       95.0     0.0  142.0  142.0  146.0         187.0         148.0   368.0
SimplePlayer         900.0           NaN    677.0        991.0         350.0      308.0       333.0      326.0    64.0  260.0  348.0  330.0         366.0         356.0   468.0
NoCheck              420.0         323.0      NaN        916.0         734.0      667.0       244.0      420.0   430.0  233.0  451.0  465.0         577.0         516.0   566.0
AlwaysCheck          992.0           9.0     84.0          NaN           0.0        0.0        11.0        0.0     0.0   11.0    0.0    0.0           8.0          16.0   132.0
HonestPlayer         991.0         650.0    266.0       1000.0           NaN      283.0       154.0  